---
jupyter:
  jupytext:
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3
    language: python
    name: python3
---

<!-- #region id="90699b8a" -->
# Table des matières
1. [La régression avec modèles linéaires dans les coefficients](#méthode-i-la-régression-avec-modèles-linéaires-dans-les-coefficients)
  1. [Linéarité ne veut pas dire trivialité!](#linéarité-ne-veut-pas-dire-trivialité)
  1. [Complexification des modèles par étapes](#complexification-des-modèles-par-étapes)
  1. [Identification des variables les plus importantes](#identification-des-variables-les-plus-importantes)
1. [La régression avec arbres décisionnels et forêts aléatoires](#méthode-ii-la-régression-avec-arbres-décisionnels-et-forêts-aléatoires)
  1. [Comment utilise-t-on un arbre pour effectuer une régression?](#comment-utilise-t-on-un-arbre-pour-effectuer-une-régression)
  1. [Le principe d'uniformisation](#le-principe-duniformisation)
  1. [Précision des prédictions](#précision-des-prédictions)
  1. [Assembler un grand nombre d'arbres ensemble](#assembler-un-grand-nombre-darbres-ensemble)
  1. [Identification des variables les plus importantes](#identification-des-variables-les-plus-importantes-1)
1. [À quoi riment toutes ces différences entre les deux méthodes?](#à-quoi-riment-toutes-ces-différences-entre-les-deux-méthodes)
    1. [Compréhension versus applications](#compréhension-versus-applications)
    1. [Modéliser les interactions](#modéliser-les-interactions)
<!-- #endregion -->

<!-- #region id="92f3e4f0" -->
Dans ce module, on s'intéresse à la régression en général. C'est l'un des outils d'analyse des données les plus
importants après la visualisation des données.

La régression est un ensemble de méthodes en statistiques et en apprentissage automatique qui est
utilisé pour analyser la relation entre une variable par rapport à une ou plusieurs autres.
C'est un sujet particulièrement important comme en témoigne le fait qu'il soit enseigné dès les
premiers cours de sciences expérimentales.

De façon générale, la régression sert à prédire une *réponse* $y$, qui est une variable
numérique continue (p. ex. un prix), en fonction de plusieurs facteurs $x_{i}$

$$y=f(x_{1}, \cdots, x_{N}, \Theta)$$

où $\Theta$ représente l'ensemble des paramètres de la fonction $f$.

La figure suivante montre un exemple typique de régression en une dimension, où
un modèle empirique de la forme

$$\text{prog} = a_{0} \exp(-a_{1} \text{days})$$

est ajusté à un ensemble de données expérimentales $(\text{days}, \text{prog})$. Les valeurs estimées des paramètres $a_0$ et $a_1$
sont affichées dans le titre de la figure.
<!-- #endregion -->

<!-- #region id="693cfa53" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/fitted-line-plot.png"  width="500" />
    <div>
    <font size="0.5">Image Source: https://online.stat.psu.edu/stat501/lesson/15/15.7/</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="f617fc5f" -->
Dans cet exemple, on modélise la variable $\text{prog}$, qui est un indice pronostique pour la récupération à long terme de patients opérés, en fonction du nombre de jours d'hospitalisation, $\text{days}$. Les résultats montrent que les chances de récupération s'amenuisent en fonction de la durée d'hospitalisation. Les données sont tirées du livre
__[*Applied Linear Regression Models*](https://www.amazon.com/Applied-Linear-Regression-Models-Student/dp/0073014664)__.


Il existe aujourd'hui un grand nombre de méthodes de régression qui sont utilisées dans différents
domaines; chacune a ses forces et ses faiblesses en fonction du type de données à analyser. Dans
les modules qui suivent nous allons nous concentrer sur deux d'entre elles largement utilisées
en recherche et dans l'industrie:


- la régression avec modèles linéaires dans les coefficients,
- la régression par forêts aléatoires.


Ces deux méthodes sont particulièrement intéressantes, car elles correspondent à deux philosophies
totalement différentes en modélisation des données. Nous allons discuter de façon générale de
chacune dans ce module. Elles seront discutées plus en détail, avec exemples à l'appui, dans
les modules qui suivent.

> À noter qu'en France, on utilise souvent l'expression recalage d'un modèle à un ensemble de données (*to fit a model to data*) pour indiquer une régression.

<!-- #endregion -->

<!-- #region id="f70e39d6" -->
# <a id=méthode-i-la-régression-avec-modèles-linéaires-dans-les-coefficients>La régression avec modèles linéaires dans les coefficients</a>
<!-- #endregion -->

<!-- #region id="d039ed7d" -->
## <a id=linéarité-ne-veut-pas-dire-trivialité>Linéarité ne veut pas dire trivialité!</a>

<!-- #endregion -->

<!-- #region id="cfc1a48b" -->
Un modèle linéaire dans ses coefficients n'implique pas qu'il soit trop simple pour être utile. Prenons l'exemple suivant d'un polynôme en $x$ de degré $N$. Chacun des coefficients $a_{i}$ est de degré 1 alors que les
termes en $x$ ont des exposants allant de $0$ à $N$. La réponse $y$ varie linéairement avec les coefficients
$a_{i}$, mais non linéairement selon la variable x.

$$y = a_{0} + a_{1}x + a_{2}x^{2} + \cdots + a_{N}x^{N}$$

La formule montre qu'un modèle de la forme $y=f(x)$ peut être hautement nonlinéaire (c.-à-d. très complexe), mais garder une expression très simple lorsqu'il est exprimé sous forme linéaire dans ses coefficients.
<!-- #endregion -->

<!-- #region id="fafb34f3" -->
## <a id=complexification-des-modèles-par-étapes>Complexification des modèles par étapes</a>
<!-- #endregion -->

<!-- #region id="fe3fd543" -->
Dans la majorité des cas où l'on veut analyser des données, le modèle mathématique sous-jacent, s'il existe, est très complexe et inconnu. Alors qu'en physique, on peut dériver exactement la formule mathématique d'un modèle, c'est rarement le cas dans les situations courantes de la vie, où beaucoup de facteurs externes influencent nos mesures.
Il est alors courant de les approximer avec des modèles linéaires dans les coefficients.

L'essence de cette approche repose sur l'observation des données analysées. Les variables mesurées
couvrent généralement de faibles plages de valeurs. Par exemple, la taille d'un adulte varie entre 1 mètre et
environ 3 mètres; il n'y a qu'un facteur 3. La température des patients varie entre 35 et 39 Celsius. Bien sûr, dans certains
cas, les variables peuvent varier sur plusieurs ordres de grandeur. Par exemple, les salaires annuels dans une
grande compagnie peuvent varier de $30\,000~~\$$ à $10\,000\,000~~\$$ avec une distribution tout sauf normale. Toutefois,
lorsqu'on prend le logarithme (en base 10) des salaires, la distribution devient normale et ses valeurs
varient entre 4,4 et 7.

En supposant que les variables mesurées varient peu autour de leur valeur moyenne, on peut approximer les
fonctions recherchées. En voici quelques exemples couramment utilisés dans le cas où il n'y a que deux
variables, $x_{1}$ et $x_{2}$.

On commence d'abord par le modèle le plus simple, et on le complexifie si nécessaire. Par exemple, si on ne garde que les termes linéaires on obtient
$$y \approx a_{0}+a_{1}x_{1}+a_{2}x_{2}$$

Si l'on ajoute ensuite le terme d'interaction $x_{1}x_{2}$ entre les variables et les termes quadratiques, on obtient
$$y \approx a_{0}+a_{1}x_{1}+a_{2}x_{2}+a_{3}x_{1}x_{2}+a_{4}x_{1}^{2}+a_{5}x_{2}^{2}$$

Il est facile de généraliser ces résultats pour un nombre arbitraire de facteurs $x_{i}$.

Les coefficients $a_{i}$ sont estimés en utilisant la méthode des moindres carrés dont il sera question dans un
prochain module. Elle n'est pas la seule permettant d'estimer les valeurs des coefficients, mais elle est
particulièrement facile à utiliser lorsque les modèles sont linéaires dans les coefficients.

<!-- #endregion -->

<!-- #region id="34b6729b" -->
## <a id=identification-des-variables-les-plus-importantes>Identification des variables les plus importantes</a>
<!-- #endregion -->

<!-- #region id="a6a40746" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/inspector-with-magnifying-glass.jpeg"  width="200" />
    <div>
    <font size="0.5">Image Source: http://clipart-library.com/clipart/1416328.htm</font>
    </div>
</div>

<!-- #endregion -->

<!-- #region id="5a636fa8" -->
Dans les formules précédentes, l'importance de chaque terme sur la réponse $y$ est déterminée par son coefficient $|a_{i}|$. Il faut toutefois que les variables $x_{i}$ aient été normalisées au préalable. On discutera de ce point dans le module sur le prétraitement des données. Les variables normalisées sont généralement de moyenne nulle et d'écart-type égal à 1. Elles sont toutes de l'ordre de 1 en valeur absolue.

On peut ainsi identifier, non seulement les variables $x_{i}$ les plus importantes dans un modèle, mais
aussi quelles sont les interactions les plus intéressantes.

Le signe de chaque coefficient facilite l'interprétation du modèle. Il n'est pas toujours facile
de le prédire à l'avance dû à la présence des interactions qui brouillent les cartes.

Par exemple, supposons que nous utilisons le modèle suivant pour analyser un jeu de données expérimentales
$(x_1, x_2, y)$ où la réponse $y$ dépend de deux variables $x_1$ et $x_2$ normalisées.

$$y = a_{0}+a_{1}x_1+a_{2}x_2+a_{2}x_1 x_2$$

Après l'entraînement du modèle de régression, on obtient

$$y = 2 + 5x_1 - 0,01x_2 - x_1 x_2$$

Selon les valeurs des coefficients, le terme en $x_1$ est 500 fois plus important que le terme en $x_2$. On peut laisser tomber ce dernier pour obtenir

$$y \approx 2 + 5x_1 - x_1 x_2$$

Bien que la variable $x_2$ prise seule a un effet négligeable sur la réponse, elle module l'effet de $x_1$ sur celle-ci. Regardons trois cas simples:


- Cas $x_2=-1$: $y \approx 2 + 6x_1$, 
- Cas $x_2=0$: $y \approx 2 + 5x_1$, 
- Cas $x_2=1$: $y \approx 2 + 4x_1$. 


Ainsi lorsque $x_2$ augmente, la réponse $y$ augmente moins rapidement avec $x_1$.

L'analyse des valeurs des coefficients montre que la réponse $y$ augmente avec la variable $x_1$, mais son effet est contrôlé par la variable $x_2$.

<!-- #endregion -->


<!-- #region id="8f105edd" -->
# <a id=méthode-ii-la-régression-avec-arbres-décisionnels-et-forêts-aléatoires>La régression avec arbres décisionnels et forêts aléatoires</a>
<!-- #endregion -->

<!-- #region id="778870e5" -->
Les forêts aléatoires sont formées en regroupant un grand nombre d'arbres décisionnels. Commençons par expliquer
comment fonctionne une régression avec un arbre décisionnel. 

> À noter que les principes expliqués dans cette section s'appliquent de façon similaire aux forêts aléatoires utilisées en classification, comme on le verra dans
un prochain module.

La figure suivante montre un exemple d'arbre décisionnel (un organigramme) où l'on veut prédire une réponse $y$ en fonction
de deux facteurs $x_{1}$ et $x_{2}$. Les cases en vert sont les feuilles de l'arbre. Les cases en bleu sont les
nœuds où des décisions se prennent et où les branches (les lignes) se séparent.
<!-- #endregion -->

<!-- #region id="c00fa99c" -->
<div align="center">
    <img src= "../images/decision-tree-example.png"  width="400" />
    <div>
    <font size="0.5">Image Source: https://medium.com/pursuitnotes/decision-tree-regression-in-6-steps-with-python-1a1c5aa2ee16/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="5e9347db" -->
## <a id=comment-utilise-t-on-un-arbre-pour-effectuer-une-régression>Comment utilise-t-on un arbre pour effectuer une régression?</a>
<!-- #endregion -->

<!-- #region id="5ab12251" -->
Il faut naviguer à travers sa structure en se déplaçant de haut en bas. Supposons que l'on essaie de prédire la réponse pour une donnée $(x_1, x_2)=(10, 100)$. Le premier nœud impose un choix; on doit se diriger vers le nœud de gauche puisque $x_1<20$. Celui-ci impose un nouveau choix; on doit se diriger vers la feuille de gauche puisque $x_2<200$. La réponse prédite par l'arbre est $y=300,5$.

<!-- #endregion -->

<!-- #region id="20e4bd9f" -->
## <a id=le-principe-duniformisation>Le principe d'uniformisation</a>
<!-- #endregion -->

<!-- #region id="df5ab945" -->
Les détails de la construction des arbres sont expliqués dans le module sur la régression par forêt aléatoire.
L'idée principale est toutefois assez simple. Lors de l'entraînement d'un arbre, on veut
assigner une feuille à chaque donnée $(x_{1}, x_{2}, y)$. Si l'ensemble d'entraînement contient $n$ données et
qu'on utilise l'arbre dans la figure précédente, les $n$ données seront réparties parmi les cinq feuilles.
Le but de l'entraînement est de bien distribuer les données dans les feuilles afin d'uniformiser les valeurs de la réponse $y$ dans celle-ci. Idéalement, les valeurs de $y$ seraient toutes les mêmes dans chaque feuille. C'est peu probable toutefois, car le nombre de données $(x_{1}, x_{2}, y)$, et donc de valeurs de $y$, est plus grand que le nombre
de feuilles. On peut améliorer l'uniformisation des données en faisant en sorte que la variance des réponses $\sigma^2(y)$ dans chaque feuille est minimale. Ainsi, à chaque noeud de l'arbre, les données doivent être distribuées afin de minimiser la variance des valeurs de $y$ dans ses deux feuilles. Si les feuilles contiennent beaucoup de données, elles deviendront des noeuds qui seront séparés à leur tour en de nouvelles feuilles. Lors de l'entraînement de l'arbre, le processus se poursuit jusqu'à ce qu'un nombre minimal de données par feuilles soit atteint.

<!-- #endregion -->

<!-- #region id="55c5018e" -->
## <a id=précision-des-prédictions>Précision des prédictions</a>

<!-- #endregion -->

<!-- #region id="55e5824d" -->
L'avantage d'uniformiser les données dans chaque feuille est de réduire les erreurs de prédiction. Imaginez un arbre avec seulement deux feuilles; il contiendra des données variées dans chaque feuille et ne prédira que deux réponses $y$. L'erreur de prédiction sera importante. Si on augmente le nombre de feuilles au point que celles-ci ne contiennent que des valeurs similaires, les prédictions seront meilleures. Dans un monde idéal, les données $(x_{1}, x_{2}, y)$ ne sont pas dégradées par du bruit. Dans la réalité, si l'on fait une même mesure $n$ fois on obtiendra un ensemble de $n$ données $(x_{1}, x_{2}, y)$ différentes. Un arbre bien conçu fera en sorte d'assigner les $n$ données à la même feuille. La variance des réponses $\sigma^2(y)$ dans la feuille sera une mesure de la variance du bruit de mesure de la réponse $y$. La moyenne des réponses $y$ dans la feuille sera une approximation de la valeur de $y$ sans bruit; c'est la réponse $y$ prédite par cette feuille. Il en résultera des erreurs de prédictions minimales.

<!-- #endregion -->

<!-- #region id="419923e6" -->
## <a id=assembler-un-grand-nombre-darbres-ensemble>Assembler un grand nombre d'arbres ensemble</a>

<!-- #endregion -->

<!-- #region id="665e6066" -->
Lorsque le nombre de variables $x_i$ est grand, les arbres tendent à devenir complexes, avec beaucoup de branches.
Cette complexité les rend fragiles, car l'ajout de quelques nouvelles données d'entraînement peut en changer la structure, ce qui affecte la précision des prédictions. Nous reviendrons sur ce point dans un prochain module.

Les systèmes experts, qui étaient à la mode dans les années 80, utilisaient des arbres décisionnels. Les décisions aux noeuds des branches étaient basées sur le jugement d'experts humains (c.-à-d. des experts du domaine). Le but était de construire un arbre permettant de prédire une réponse aussi bien qu'un humain. Celle-ci pouvait être une valeur numérique quelconque (le prix d'une maison) ou une classe (une décision de traitement médical ou la suggestion de faire une nouvelle série d'examens). Les décisions d'un système expert étaient ensuite utilisées dans un autre système expert simulant le jugement d'un médecin dans une autre spécialité par exemple. Ce type d'intelligence artificielle n'a pas porté ses fruits, car le jugement humain est souvent basé sur quelques règles et non les milliers qu'il aurait fallu pour prendre une bonne décision. De plus, la structure même des arbres décisionnels les rend fragiles comme on vient de le voir.

Une solution à ce problème fut de rassembler les prédictions d'un grand nombre d'experts pouvant faire des erreurs. Cela a mené à la méthode des forêts aléatoires dont il sera amplement question dans cette formation. La figure suivante montre un exemple de forêt aléatoire composée d'un grand nombre d'arbres décisionnels (600). Elle rassemble les prédictions de chacun et en calcule la moyenne; c'est la prédiction, ou la réponse $y$, de la forêt.
<!-- #endregion -->

<!-- #region id="eee966d3" -->
<div align="center">
    <img src= "../images/random-forest-example.png"  width="400" />
    <div>
    <font size="0.5">Image Source: https://www.keboola.com/blog/random-forest-regression/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="9954a0ea" -->
### <a id=identification-des-variables-les-plus-importantes-1>Identification des variables les plus importantes</a>
<!-- #endregion -->

<!-- #region id="22fa7560" -->
<div align="center">
    <img src= "../images/inspector-with-magnifying-glass.jpeg"  width="200" />
    <div>
    <font size="0.5">Image Source: http://clipart-library.com/clipart/1416328.htm</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="85589e60" -->
Dans un arbre décisionnel, les variables $x_{i}$ d'intérêt sont les plus utilisées pour prendre des décisions à
travers ses branches. Dans une forêt aléatoire, la distribution des variables d'intérêt de chaque arbre, compilée pour la forêt dans son ensemble fournit la liste des variables les plus importantes. Les fonctions de régression de Scikit-learn permettent de les déterminer.
<!-- #endregion -->


<!-- #region id="82f612ec" -->
# <a id=à-quoi-riment-toutes-ces-différences-entre-les-deux-méthodes>À quoi riment toutes ces différences entre les deux méthodes?</a>
<!-- #endregion -->

<!-- #region id="b121930d" -->
### <a id=compréhension-versus-applications>Compréhension versus applications</a>
<!-- #endregion -->

<!-- #region id="020a28c1" -->
La première méthode est analytique, prévisible et bien polie. La seconde est heuristique et chaotique.
Ces différences sont fondamentales.

En sciences et génie, on essaie de modéliser analytiquement les phénomènes (physiques, chimiques, etc.) que
l'on étudie puis on essaie de résoudre les équations. La plupart du temps, elles sont trop complexes
pour obtenir des solutions exactes. Comme on l'a vu, on fait alors des approximations menant à des modèles
linéaires dans les coefficients. Il faut faire la sélection
des variables pertinentes $x_{i}$, décider des interactions $x_{i}x_{j}$ importantes, etc. Ça demande une
bonne connaissance du domaine. On ajuste ensuite le modèle aux données expérimentales. Si les résultats
ne sont pas très bons, il faut penser à inclure de nouvelles variables et interactions. Lesquelles? Cela
nécessite une forte expertise du domaine, que peu de gens ont. Si, à la fin, notre modèle explique $90~\%$ des mesures expérimentales, on peut alors être très satisfaits. Après tout, on est probablement les premiers humains dans toute
l'Histoire à comprendre les bases d'un nouveau phénomène découvert il n'y a pas longtemps.

Toutefois, il arrive souvent que l'on souhaite faire de bonnes prédictions plutôt que d’en comprendre les
détails théoriques sous-jacents. On inclut alors un très grand nombre de variables $x_{i}$ plus ou moins hétéroclites
et on laisse le régresseur, dans ce cas-ci une forêt aléatoire, faire les prédictions. Dans ce cas-ci, l'expert occupe un rôle moins central, car l'algorithme détermine empiriquement lui-même les interactions entre les variables.
Si à la fin le modèle prédit $98~\%$ des mesures expérimentales alors tout le monde est content et les affaires vont bien. On peut intégrer cette application prédictive dans un nouveau produit.

<!-- #endregion -->

<!-- #region id="eeeb146e" -->
### <a id=modéliser-les-interactions>Modéliser les interactions</a>
<!-- #endregion -->

<!-- #region id="e6f526d1" -->
Le monde est complexe. Une des difficultés à le modéliser est la multitude d'interactions possibles entre
les variables impliquées. Il faut identifier les plus importantes afin de les introduire
dans un modèle analytique. Ce n'est pas facile.

L'apport des forêts aléatoires est immense ici. On peut construire des arbres décisionnels
avec beaucoup de facteurs $x_{i}$ sans avoir à inclure de nouveaux facteurs pour modéliser des
interactions entre les facteurs déjà présents, par exemple $x_{2}x_{3}$. En effet, la logique
des choix effectués en naviguant à travers un arbre décisionnel
implique déjà des interactions multiples. Pas besoin de les préciser à l'avance! Il peut y en
avoir un très grand nombre si l'arbre est suffisamment complexe. La réunion de tous les arbres en
une forêt permet de modéliser les données d'une façon hautement non linéaire. Cela va bien au-delà de
ce que peut faire un modèle analytique pour lequel il faut spécifier à l'avance tous les termes
d'intérêt, comme dans la méthode I.

Les forêts aléatoires, et leurs variantes, sont parmi les meilleures méthodes de régression et battent souvent
en performances les réseaux de neurones dans les compétitions internationales de type Kaggle.

Cette discussion s'applique autant aux méthodes de régression et de classification que l'on verra dans cette formation.
Nous le rappellerons à plusieurs reprises dans les futurs modules.
<!-- #endregion -->
